In [23]:
import pandas as pd

In [24]:
biometric_df = pd.read_csv("../data/processed/interim/biometric_raw_merged.csv")

In [25]:
biometric_df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,01-03-2025,Haryana,Mahendragarh,123029,280,577
1,01-03-2025,Bihar,Madhepura,852121,144,369
2,01-03-2025,Jammu and Kashmir,Punch,185101,643,1091
3,01-03-2025,Bihar,Bhojpur,802158,256,980
4,01-03-2025,Tamil Nadu,Madurai,625514,271,815


In [26]:
biometric_df.columns


Index(['date', 'state', 'district', 'pincode', 'bio_age_5_17', 'bio_age_17_'], dtype='object')

In [27]:
biometric_df = biometric_df.rename(columns={
    "bio_age_17_": "bio_age_17_plus"
})


In [28]:
biometric_df.shape

(1861108, 6)

In [29]:
biometric_df["state"] = biometric_df["state"].str.strip().str.title()

In [30]:
biometric_df["district"] = biometric_df["district"].str.strip().str.title()

In [31]:
biometric_df["date"] = pd.to_datetime(biometric_df["date"], dayfirst=True)

In [32]:
biometric_df.dtypes

date               datetime64[ns]
state                      object
district                   object
pincode                     int64
bio_age_5_17                int64
bio_age_17_plus             int64
dtype: object

In [33]:
biometric_df = biometric_df.drop_duplicates()


In [34]:
biometric_df.shape

(1766159, 6)

In [35]:
biometric_df[
    biometric_df["district"].isin(["100000", 100000]) |
    biometric_df["state"].isin(["100000", 100000])
].shape[0]


0

In [36]:
biometric_df.shape

(1766159, 6)

In [37]:
biometric_df.isnull().sum()


date               0
state              0
district           0
pincode            0
bio_age_5_17       0
bio_age_17_plus    0
dtype: int64

In [38]:
biometric_df["state"].value_counts()


state
Tamil Nadu                                  174934
Andhra Pradesh                              160231
Uttar Pradesh                               147138
Maharashtra                                 143609
Karnataka                                   135773
West Bengal                                 125316
Kerala                                       93951
Gujarat                                      84630
Odisha                                       83191
Bihar                                        78078
Telangana                                    77850
Rajasthan                                    76698
Madhya Pradesh                               66020
Punjab                                       46414
Assam                                        44418
Jharkhand                                    35044
Chhattisgarh                                 30048
Himachal Pradesh                             28723
Haryana                                      25083
Uttarakhand              

In [39]:
biometric_state_mapping = {
    "Orissa": "Odisha",
    "Pondicherry": "Puducherry",

    "West Bangal": "West Bengal",
    "Westbengal": "West Bengal",
    "West  Bengal": "West Bengal",

    "Jammu & Kashmir": "Jammu And Kashmir",

    "Chhatisgarh": "Chhattisgarh",
    "Uttaranchal": "Uttarakhand",

    "Tamilnadu": "Tamil Nadu",

    "Dadra And Nagar Haveli": "Dadra And Nagar Haveli And Daman And Diu",
    "Dadra & Nagar Haveli": "Dadra And Nagar Haveli And Daman And Diu",
    "Daman And Diu": "Dadra And Nagar Haveli And Daman And Diu",
    "Daman & Diu": "Dadra And Nagar Haveli And Daman And Diu",

    "Andaman & Nicobar Islands": "Andaman And Nicobar Islands"
}


In [40]:
biometric_df["state"] = biometric_df["state"].replace(biometric_state_mapping)


In [41]:
# -----------------------------
# Fix Hyderabad → Telangana
# -----------------------------

mask = (
    biometric_df["district"].astype(str).str.strip().str.title() == "Hyderabad"
)

affected_rows = mask.sum()

biometric_df.loc[mask, "state"] = "Telangana"

print(f"✔ Hyderabad correction applied to {affected_rows} rows")


✔ Hyderabad correction applied to 8831 rows


In [42]:
biometric_df["state"].value_counts()


state
Tamil Nadu                                  174935
Andhra Pradesh                              156043
Uttar Pradesh                               147138
Maharashtra                                 143609
Karnataka                                   135773
West Bengal                                 125413
Odisha                                       95958
Kerala                                       93951
Gujarat                                      84630
Telangana                                    82038
Bihar                                        78078
Rajasthan                                    76698
Madhya Pradesh                               66020
Punjab                                       46414
Assam                                        44418
Jharkhand                                    35044
Chhattisgarh                                 30053
Himachal Pradesh                             28723
Haryana                                      25083
Uttarakhand              

In [43]:
before = len(biometric_df)

biometric_df = biometric_df[
    (biometric_df["state"] != "<unset>") &
    (biometric_df["district"] != "<unset>") &
    (biometric_df["pincode"] != "<unset>")
]

after = len(biometric_df)

print(f"Removed {before - after} rows with <unset> in key columns")


Removed 0 rows with <unset> in key columns


In [44]:
biometric_df.to_csv(
    "../data/processed/cleaned/biometric_clean.csv",
    index=False
)
